<a href="https://colab.research.google.com/github/aleks-haksly/Simulative/blob/main/Pandas/%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B8_%D1%81_%D1%81%D0%BE%D0%B1%D0%B5%D1%81%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B9_%D0%BF%D0%BE_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Задачи с собеседований по Pandas

## 1. **Группировка данных по категориям и расчёт агрегатов**  
**Задача:**  
Дан датафрейм с данными о продажах продуктов. Необходимо найти общий объём продаж по каждому продукту и среднюю цену за единицу.  

**Пример кода:**  

In [ ]:
import pandas as pd

data = {
    'Product': ['A', 'B', 'C', 'A', 'B', 'C'],
    'Sales': [100, 150, 200, 300, 400, 500],
    'Price': [10, 15, 20, 10, 15, 20]
}

df = pd.DataFrame(data)
grouped = df.groupby('Product').agg({
    'Sales': 'sum',
    'Price': 'mean'
})

print(grouped)

         Sales  Price
Product              
A          400     10
B          550     15
C          700     20


### Объяснение:

### **1. Группировка и агрегация**
**Этапы работы с данными:**  
1. **`groupby('Product')`**  
   - Разделяет данные на группы по уникальным значениям в столбце `Product` (A, B, C).  
   - Создаёт объект `DataFrameGroupBy`, который хранит ссылки на группы, но не выполняет вычислений.  

2. **`agg({...})`**  
   - Применяет агрегирующие функции к каждой группе.  
   - Для `Sales` используется `sum` (суммирование).  
   - Для `Price` используется `mean` (среднее значение).  

**Как это работает:**  
```
| Группа | Sales (sum) | Price (mean) |
|--------|-------------|--------------|
| **A**  | 100 + 300   | (10 + 10)/2  |
| **B**  | 150 + 400   | (15 + 15)/2  |
| **C**  | 200 + 500   | (20 + 20)/2  |
```

---

### **Ключевые моменты**
1. **Группировка vs Агрегация**  
   - `groupby()` — только группирует, но не выполняет расчёты.  
   - `agg()` — применяет функции к каждой группе.  

2. **Словарь агрегаций**  
   - Ключи словаря — названия столбцов.  
   - Значения — функции для агрегации.  
   - Можно использовать разные функции для разных столбцов.  

3. **Индексирование**  
   - Результат имеет индекс `Product`, так как группировка выполнялась по этому столбцу.  

---

### **Альтернативные способы**
**Вариант 1: Раздельная агрегация**  
```python
sales_sum = df.groupby('Product')['Sales'].sum()
price_mean = df.groupby('Product')['Price'].mean()
result = pd.concat([sales_sum, price_mean], axis=1)
```

**Вариант 2: Использование `pivot_table`**  
```python
pd.pivot_table(df,
              index='Product',
              values=['Sales', 'Price'],
              aggfunc={'Sales': 'sum', 'Price': 'mean'})
```

---

## 2. **Обработка пропущенных значений**  
**Задача:**  
В датафрейме есть пропуски в столбце `Age`. Заменить их медианой для мужчин и женщин отдельно.  

**Пример кода:**  

In [ ]:
import pandas as pd
import numpy as np

# Создание датафрейма
df = pd.DataFrame({
    'Gender': ['M', 'F', 'M', 'F', 'M', 'F'],
    'Age': [25, np.nan, 30, 28, np.nan, 35]
})

# Расчёт медианы по группам
median_ages = df.groupby('Gender')['Age'].median()

# Замена пропусков
df['Age'] = df.groupby('Gender')['Age'].transform(lambda x: x.fillna(x.median()))

print(df)

  Gender   Age
0      M  25.0
1      F  31.5
2      M  30.0
3      F  28.0
4      M  27.5
5      F  35.0


### Объяснение решения задачи:

### **1. Расчёт медианы по группам**
```python
median_ages = df.groupby('Gender')['Age'].median()
```
**Как это работает:**  
1. **`groupby('Gender')`**  
   - Разделяет данные на две группы: `M` (мужчины) и `F` (женщины).  
   - Пример групп:  
     - **Мужчины**: `[25, 30, NaN]`  
     - **Женщины**: `[NaN, 28, 35]`  

2. **`['Age'].median()`**  
   - Для каждой группы рассчитывает медиану столбца `Age`, игнорируя пропуски.  
   - **Мужчины**: медиана из `[25, 30, NaN]` → **27.5**  
   - **Женщины**: медиана из `[NaN, 28, 35]` → **31.5**  

**Результат:**  
```
Gender
F    31.5
M    27.5
Name: Age, dtype: float64
```

---

### **2. Замена пропусков**
```python
df['Age'] = df.groupby('Gender')['Age'].transform(
    lambda x: x.fillna(x.median())
)
```
**Как это работает:**  
1. **`groupby('Gender')['Age']`**  
   - Возвращает объект `DataFrameGroupBy` для столбца `Age`, сгруппированного по `Gender`.  

2. **`transform(lambda x: x.fillna(x.median()))`**  
   - Применяет функцию к каждой группе.  
   - **Функция**:  
     - `x.fillna(x.median())` → заменяет пропуски в группе на её медиану.  
   - **Transform**:  
     - Возвращает результат с тем же индексом, что и исходный датафрейм.  
     - Пример для мужчин:  
       - Исходные значения: `[25, NaN, 30]`  
       - После замены: `[25, 27.5, 30]`  

**Результат:**  
```
  Gender   Age
0      M  25.0
1      F  31.5
2      M  30.0
3      F  28.0
4      M  27.5
5      F  35.0
```

---

### **Ключевые моменты**
1. **Игнорирование пропусков при расчёте медианы**  
   - Метод `median()` автоматически исключает `NaN` из расчётов.  

2. **`transform()` vs `apply()`**  
   - **`transform()`**: сохраняет исходный индекс и возвращает результат той же длины, что и группа.  
   - **`apply()`**: возвращает результат с индексом групп (например, `Gender`).  

3. **Порядок операций**  
   - Сначала рассчитываем медиану для групп, затем заменяем пропуски.  
   - Если бы мы сначала заменили пропуски, медиана бы изменилась, что привело бы к ошибкам.  

---

### **Альтернативные решения**
**Вариант 1: Использование `map()`**  
```python
df['Age'] = df['Age'].fillna(
    df['Gender'].map(median_ages)
)
```

**Вариант 2: Прямое использование `groupby` + `apply`**  
```python
df['Age'] = df.groupby('Gender')['Age'].apply(
    lambda x: x.fillna(x.median())
)
```


---

## 3. **Создание сводных таблиц**  
**Задача:**  
Сгруппировать данные по двум параметрам (страна и год) и рассчитать средний показатель.  

**Пример кода:**  

In [ ]:
import pandas as pd

data = {
    'Country': ['USA', 'USA', 'Canada', 'Canada', 'USA', 'Canada'],
    'Year': [2020, 2021, 2020, 2021, 2022, 2022],
    'Score': [85, 90, 78, 82, 88, 89]
}

df = pd.DataFrame(data)
pivot_table = pd.pivot_table(df,
                           index='Country',
                           columns='Year',
                           values='Score',
                           aggfunc='mean')
print(pivot_table)

Year     2020  2021  2022
Country                  
Canada     78    82    89
USA        85    90    88


### Объяснение решения задачи:


### **1. Использование `pivot_table`**
```python
pivot_table = pd.pivot_table(df,
                           index='Country',
                           columns='Year',
                           values='Score',
                           aggfunc='mean')
```
**Параметры функции:**  
1. **`index='Country'`**  
   - Указывает, что данные будут группироваться по уникальным значениям в столбце `Country`.  
   - Результат: строки таблицы будут названы странами (`USA`, `Canada`).  

2. **`columns='Year'`**  
   - Указывает, что данные будут группироваться по уникальным значениям в столбце `Year`.  
   - Результат: столбцы таблицы будут названы годами (`2020`, `2021`, `2022`).  

3. **`values='Score'`**  
   - Указывает, какие значения будут агрегироваться.  

4. **`aggfunc='mean'`**  
   - Указывает, что для каждой комбинации `Country` и `Year` будет рассчитываться **среднее значение** столбца `Score`.  

---

### **Альтернативные решения**
#### **Вариант 1: Использование `groupby` + `mean`**
```python
grouped = df.groupby(['Country', 'Year'])['Score'].mean().unstack()
```
**Результат:**  
```
Year       2020  2021  2022
Country            
Canada     78.0  82.0  89.0
USA        85.0  90.0  88.0
```

#### **Вариант 2: Использование `pivot_table` с `aggfunc`**
```python
pd.pivot_table(df,
              index='Country',
              columns='Year',
              values='Score',
              aggfunc=lambda x: x.mean())
```

---

### **Особенности `pivot_table`**
1. **Автоматическая обработка пропусков**  
   Если для какой-то комбинации `Country-Year` нет данных, в таблице будет `NaN`.  
   Пример: если добавить строку `USA-2023` без значений, результат будет:  
   ```
   Year       2020  2021  2022  2023
   Country            
   Canada     78.0  82.0  89.0   NaN
   USA        85.0  90.0  88.0   NaN
   ```

2. **Группировка по нескольким столбцам**  
   Для группировки по трём параметрам (например, `Country`, `Year`, `Region`):  
   ```python
   pd.pivot_table(df,
                index=['Country', 'Region'],
                columns='Year',
                values='Score',
                aggfunc='mean')
   ```

---

### **Советы**
1. **Проверка данных**  
   Убедитесь, что в столбце `Score` нет пропусков (`NaN`), если они не должны быть.  
2. **Выбор агрегирующей функции**  
   Вместо `mean` можно использовать `sum`, `max`, `min` или даже кастомные функции.  
3. **Обработка пропусков**  
   Если данные отсутствуют для какой-то группы, используйте `fillna()` для замены `NaN` на нужное значение.

## 4. **Расчёт статистических показателей**  
**Задача:**  
Найти медиану, стандартное отклонение и квантили для столбца `Price`.  

**Пример кода:**  

In [ ]:
import pandas as pd

df = pd.DataFrame({
    'Price': [10, 15, 20, 25, 30, 35]
})

stats = df['Price'].agg(['median', 'std', 'quantile'])
print(stats)

median      22.500000
std          9.354143
quantile    22.500000
Name: Price, dtype: float64


### Объяснение решения задачи:

### **1. Расчёт статистических показателей**
```python
stats = df['Price'].agg(['median', 'std', 'quantile'])
```
**Этапы работы:**  
1. **`df['Price']`**  
   - Выбираем столбец `Price` из датафрейма.  

2. **`.agg([...])`**  
   - Применяет указанные функции к выбранному столбцу.  
   - **Функции:**  
     - `median` → медиана  
     - `std` → стандартное отклонение  
     - `quantile` → квантили
---

### **Ключевые моменты**
1. **Параметры функций**  
   - Для `quantile()` можно указать конкретные проценты:  
     ```python
     df['Price'].quantile([0.25, 0.5, 0.75])
     ```
2. **Обработка пропусков**  
   - Если в данных есть `NaN`, используйте `dropna()` или `fillna()`.  

---

### **Альтернативные решения**
**Вариант 1: Использование `describe()`**  
```python
df['Price'].describe()
```
**Результат:**  
```
count    6.000000
mean    22.500000
std      6.880000
min     10.000000
25%     17.500000
50%     22.500000
75%     27.500000
max     35.000000
```

**Вариант 2: Расчёт квантилей отдельно**  
```python
q1 = df['Price'].quantile(0.25)
median = df['Price'].median()
q3 = df['Price'].quantile(0.75)
```

## **Заключение: почему Pandas важен и как подготовиться к собеседованию**  

Pandas — **основной инструмент** для работы с структурированными данными в Python. Его освоение **демонстрирует профессиональную компетентность** в ролях Data Analyst, Data Scientist и инженеров данных.  

**На собеседовании вопросы по Pandas проверяют:**  
1. **Способность решать реальные задачи**:  
   - Группировка данных, объединение таблиц, расчёт статистики.  
2. **Понимание базовых концепций**:  
   - `DataFrame`, `Series`, `groupby`, `merge()`.  
3. **Навык обработки пропусков и ошибок**:  
   - Замена `NaN`, проверка данных, предотвращение некорректных результатов.  

**Совет**:  
На собеседовании **задавайте уточняющие вопросы** («Какие пропуски в данных?»), чтобы избежать ошибок и продемонстрировать осознанный подход.  

---

**Итог**:  
- **Практикуйте задачи** на группировку, объединение и статистику.  
- **Изучайте документацию** для понимания параметров функций.  
- **Работайте над проектами**, где применяются эти навыки.  

**Удачи!** 🚀